# 1. 主从复制准备

一般常规主从复制在不同的机器上，并且监听的端口均为默认的3306。  
本文主从复制技术环境以单数据库多实例来讲解学习，上文已经写了怎么配置mysql多实例。

## 1.1 定义服务器角色

主库（mysql master）:ip为192.168.0.107 port为3306

从库1（mysql slave）:ip为192.168.0.107 port为3307

从库2 （mysql slave）:ip为192.168.0.107 port为3308

从库3 (mysql salve)：ip为192.168.0.107，port 为3309

## 1.2 数据库环境准备

- 具备单机单数据库多实例的环境。
- 或两台服务器每个机器一个数据库环境。

## 1.3 数据库读法的约定

- 主库，master
- 从库，slave

# 2. 主库上执行的操作

### 2.1 设置server-id值，并且开启binlog参数

MySQL的主从复制的关键因素就是binlog日志是否开启，所有首先打开bin-log日志，按如下两个参数修改。重启服务器生效。

注意：
1. server-id, log_bin参数要放在my.cnf中的[mysqld]模块下，否则会出错。
2. 一般server-id的值使用服务器ip地址的最后8位。如107，目的是避免不同机器或实例ID重复（不适合多实例）

查看主库log_bin是否开启：

看3306目录下有没有日志，有日志生成则成功开启：

### 2.2 创建用于同步的账号

创建用户rep，用户与从库 主从复制。

提示：
1. replication slave同步的必须权限，不要授权all。
2. 表示所有库所有表。
3. 'rep'@'192.168.0.%' rep为同步账号，192.168.0.%授权主机段。

### 2.3 对数据库锁表只读

生产环境操作主从复制需要申请停机时间，数据量很大锁表会影响业务可以在备份数据库数据的时候让--master-data的值等于1。

mysql5.1.X版本的锁表是flush tables，mysql5.5.X及以上版本是flush table。锁表不要退出窗口。

提示：  
这个锁表命令的时间，在不同引擎的情况下,会受到下面参数的限制，超过设置的时间会自动解锁。

- Interaction_timeout = 28800
- wait_timeout = 28800

默认情况下的时间长为：28800秒(8小时)，主要看interactive_timeout和wait_timeout这两个参数。

### 2.4 查看主库的状态 show master status

查看主库的状态及当前binlog日志文件名和二进制binlog日志偏移量
show master status;命令显示的数据要记录下来，后面的从库复制时是从这个位置开始的。

### 2.5 导出数据库数据

导出数据库数据，如果数据量很大并且允许停机可以停库直接将主库的所有数据文件打包不用mysqldump，并拷贝到从库的数据文件目录。

- -A参数表示所有数据库，
- -B参数增加use db和drop语句。
- --master-data参数最小化备份mysql数据库可以帮助从库直接找到binlog的位置。

为了保证导库期间，数据库没有数据写入，可以再检查主库状态信息，如果和之前查看的不对就出问题了没锁住表。

备份数据后解锁主库恢复可写；

### 2.6 把主库导出的mysql数据迁移到从库

可以就用scp将备份的日志文件传给从服务器就可以，这里使用多实例就不讲解了。

-----

# 3. 从库上执行的操作

### 3.1 设置server-id值，并关闭binlog参数

数据库的server-id值是唯一的，这里的server-id要和主库及其他从库不同，并注释掉从库里的binlog参数配置。

有两种情况需要打开log-bin记录数据库更新的sql语句

1. 级联同步A->B->C;那中间的B就要开启log-bin。如下所示：是从库开启binlog的配置

这个地方看一下expire_logs_days，定期删除binlog日志文件，例如把参数的值设为7，就会把最近7天以前的binlog日志文件删除。

2. 在从库做数据备份数据备份必须要有全备和binlog日志，才是完整的备份。

重启从库：

### 3.2 还原主库导出的数据到从库

### 3.3 登录从库配置同步参数

### 3.4 启动从库同步开关

启动从库同步开关并查看同步状态，关闭主从复制stop slave;

判断复制是否搭建成功就看如下IO和SQL两个线程是否显示yes状态

### 3.5 测试复制结果

在主数据库上创建一个test_slave库:

在从库上检查是否有test_slave库。

提示：  
Master-info和relay-info中继日志都会自动记录主库binlog更新数据的位置，例如我们在主库上创建一个库，master-info和relay-info中记录主库binlog日志的位置就由107变成了202。

----

# 4. 总结：MySQL主从同步配置步骤

1. 准备两台数据库环境，或者单台多实例环境，能否正常启动和登录。
2. 配置my.cnf文件，主库配置log-bin和server-id功能。注意：配置参数后要重启生效。
3. 登录主库添加用于从库连接主库的账号列如：rep并授权replication slave同步的权限。
4. 登录主库，整库锁表flush table with read lock(窗口关闭后及失效，超时参数到了也会失效)；然后show master status;查看binlog的位置状态。
5. 新开窗口，linux命令行备份或导出原有的数据库数据，并拷贝到从库所在的服务器目录。如果数据量很大并且允许停机，可以停机打包将数据直接拷贝到从库的数据库目录中，而不用mysqldump。
6. 解锁主库 unlock tables。
7. 把主库导出的原有数据恢复到从库。
8. 根据主库的show master status查看binlog的位置状态，在从库执行change master to ...语句，配置主库的IP，端口号，用于主从同步的用户账号，密码，binlog日志名称，偏移量位置。
10. 从库开启同步开关，start slave。
11. 从库show slave stutes\G.检查同步状态，并在主库进行更新测试。

# 5. 回顾mysql主从复制原理要点
1. 异步方式同步。

2. 逻辑同步模式，多种模式，默认是通过sql语句执行。

3. 主库通过记录binlog实现对从库的同步。binlog是记录数据库的更新语句。

4. 主库1个IO线程，从库由1个IO线程和2个SQL线程来完成的。

5. 从库关键文件master.info记录了change master的所有信息，relay-log被称为中继日志记录了主库IO线程发过来的binlog日志，relay-info是SQL线程把binlog应用到数据库里的一个位置点功能。

6. 如果从库还想级联从库，需要打开log-bin和log-slave-updates参数。

# 6.生产环境快速配置mysql主从复制方案
1. 安装好要配置从库的数据库，配置好log-bin和server-id参数。
2. 无需配置主库my.cnf文件，主库的log-bin和server-id 参数默认就是最好的。
3. 登录主库增加用于从库连接主库同步的账户例如：rep并授权replication slave 同步的权限。
4. 在半夜使用mysqldump带--master-data=1全备数据并恢复到从库，在从库配置同步参数是不用配置二进制文件的名称和主从同步的位置。
5. 在从库执行change master to..语句，无需binlog文件及对应位置点。
6. 从库开启同步开关，start slave。
7. 从库show master status\G 检查同步的状态，并在主库进行更新测试。

# 7.用--master-data=1方式全备数据
生产环境快速配置mysql主从复制方案实战；

## 7.1 主库上的操作
生产环境快速配置主从复制的方法不用锁表。

### 7.1.1 备份数据
用--master-data=1备份数据，在从库change master时不用指定binlog日志文件和位置点。主库不用查看master的状态。只需开启binlog，并创建用于主从同步的账户。

## 7.2 从库的操作

### 7.2.1 设置server-id值并关闭binlog参数

server-id = 3

### 7.2.2 还原主库导出的数据到从库

### 7.2.3 登录从库配置同步参数

### 7.2.4 启动从库主从复制

## 7.3 测试

主库上操作数据库，查看从库是否同步。

# 8. 生产环境主从同步配置注意事项
1. 申请设备资源。用来做从库服务器。从库服务器和主库差距不要太大。
2. 撰写方案文档和实施步骤。

例如你的服务器只有主库，而且已经跑了生产线上的应用了，现在由于业务需要第一次做从库，此时可能需要和公司申请停机维护时间（要确认这个时间内可以做一次全备），即在用户访问量最小，且不影响内部其他业务运转的时间点来停机（包括锁表）配置主从复制，一般都是凌晨进行。注意：停机（锁表，挺库）的最小时间段，为锁表后备份的时间，也就是说无需等待主从配置最好。

# 9.全备数据及获取binlog位置
也可以不申请停机时间，在定时任务备份时每天夜里，每天夜里服务小的时刻定时备份如：模拟主从同步的步骤，获取到全备及全备过程中binlog位置的信息或者直接用--master-data=1 参数解决。

1. 方法一：不用--master-date参数备份数据

不用--master-data参数或者--master-data=2备份数据，需要全备数据并且记录binlog更新的位置

2. 方法二：用--master-data=1备份数据

用--master-data=1备份数据值对数据进行全备即可，不需要记录binlog的位置

# 10. 查看线程的状态